# Hidden Markov Models (HMMs) for Song Lyrics

### Read in data and prepare

In [13]:
import pandas as pd

songs = pd.read_csv('billboard.csv', encoding='latin1')
songs.shape
songs = songs[(songs['Lyrics'].notnull()) & (songs['Lyrics'] != '  ')]
songs.shape
songs['word_count'] = songs['Lyrics'].apply(lambda s: len(s.split()))
songs['Decade'] = songs['Year'].apply(lambda y: str(y//10) + "0's")
songs = songs[songs['word_count'] > 1]
songs.shape

(4831, 8)

### Create states and transitions files

In [8]:
def create_states(year):
    lyrics = songs[songs['Decade'] == str(year) + "'s"]['Lyrics']
    words = ' '.join(lyrics)
    lst = words.split(" ")
    lst = ['-' if x=='' else x for x in lst]
    unique = sorted(set(lst))
    newfile = open(str(year) + "_states.txt","w")
    for word in unique:
        newfile.write(word + "\t" + word + "\t" + "1.0" + "\n")
    newfile.close()
    return lst, unique

In [9]:
import numpy as np

def create_transitions(year, lst, unique):
    string = lst
    #matrix holding transition probabilities
    matrix = np.zeros((len(unique),len(unique)))
    finalMatrix = np.zeros((len(unique),len(unique)))
    #dictionary of words to index
    alphabet = unique
    #loop through string
    for i in range(len(string)-2):
        #for each character, look at the next character and update the matrix
        index1 = alphabet.index(string[i])
        index2 = alphabet.index(string[i+1])
        matrix[index1][index2] += 1
    #divide each cell by the row total
    i = 0
    for row in matrix:
        rowsum = np.sum(row)
        j = 0
        for col in row:
            if rowsum != 0:
                finalMatrix[i][j] = col/rowsum
            else:
                finalMatrix[i][j] = 0.0
            j += 1
        i += 1

    newfile = open(str(year) + "_transitions.txt","w")
    newfile.write("\t")
    for char in alphabet:
        newfile.write(char + "\t")
    newfile.write("\n")

    i=0
    for row in finalMatrix:
        newfile.write(alphabet[i] + "\t")
        for col in row:
            newfile.write(str(col))
            newfile.write("\t")
        newfile.write("\n")
        i += 1
    newfile.close()
    
    return finalMatrix, alphabet


### Attempt to use HMM to predict decade of song
This was unsuccessful due to the time complexity of HMM.

In [1]:
#from HMM import HMM
# h = HMM()
# h.fit('1960_states.txt', '1960_transitions.txt')

In [2]:
# h.predict('1960_observation.txt')
# print(math.exp(h.score))

# Generating Lyrics

In [10]:
import random 

def generate_bad_lyrics(finalMatrix, alphabet):
    currIndex = random.randint(0,len(finalMatrix)-1)
    LENGTH_SONG = 125
    song = ""
    for i in range(LENGTH_SONG):
        if i % 7 == 0:
            song += "\n"
        song += alphabet[currIndex] + " "
        currIndex = finalMatrix[currIndex].argmax(axis=0)
    return song

In [11]:
def generate_okay_lyrics(finalMatrix, alphabet):
    currIndex = random.randint(0,len(finalMatrix)-1)
    LENGTH_SONG = 125
    song = ""
    
    for i in range(LENGTH_SONG):
        if i % 7 == 0:
            song += "\n"
        song += alphabet[currIndex] + " "
        currIndex = finalMatrix[currIndex].argsort()[-3:][random.randint(0,2)]
    return song


### Generating Lyrics Per Decade

In [15]:
#1960s
print("1960s")
list_1960, unique_1960 = create_states(1960)
final_1960, alpha_1960 = create_transitions(1960, list_1960, unique_1960)

1960s


In [18]:
print("BAD", generate_bad_lyrics(final_1960, alpha_1960))
print()
print("OKAY", generate_okay_lyrics(final_1960, alpha_1960))

BAD 
turnip green grass is a little bit 
me and i love you know that 
i love you know that i love 
you know that i love you know 
that i love you know that i 
love you know that i love you 
know that i love you know that 
i love you know that i love 
you know that i love you know 
that i love you know that i 
love you know that i love you 
know that i love you know that 
i love you know that i love 
you know that i love you know 
that i love you know that i 
love you know that i love you 
know that i love you know that 
i love you know that i 

OKAY 
widowed forsee fortress forsake me down and 
the way down and im proud easy 
its all right yeah and the world 
is a gas but you baby you 
baby baby i got me i cant 
you and you can see you i 
dont you i dont want my baby 
im so good lovin feelin low enough 
it was to the world needs now 
you can do the world and i 
dont let your eyes are my love 
me down in my heart is in 
your mind if you know you know 
i got you baby baby baby baby 
im p

In [19]:
#1970s
print("1970s")
list_1970, unique_1970 = create_states(1970)
final_1970, alpha_1970 = create_transitions(1970, list_1970, unique_1970)

1970s


In [20]:
print("BAD", generate_bad_lyrics(final_1970, alpha_1970))
print()
print("OKAY", generate_okay_lyrics(final_1970, alpha_1970))

BAD 
blastin drinkin if you know that i 
love you know that i love you 
know that i love you know that 
i love you know that i love 
you know that i love you know 
that i love you know that i 
love you know that i love you 
know that i love you know that 
i love you know that i love 
you know that i love you know 
that i love you know that i 
love you know that i love you 
know that i love you know that 
i love you know that i love 
you know that i love you know 
that i love you know that i 
love you know that i love you 
know that i love you know 

OKAY 
drawn in the world to me to 
be there was made a love me 
you and the world i dont know 
i love me you i dont you 
i dont wanna be with your mind 
if you and you got a song 
sing it all i love me you 
know that youre a love to me 
and you got the way to be 
there to you i love me to 
you can make you and the world 
of you i dont know how do 
you know what a friend - you 
got no time i dont stop dont 
know what you can see me i 
love y

In [21]:
#1980s
print("1980s")
list_1980, unique_1980 = create_states(1980)
final_1980, alpha_1980 = create_transitions(1980, list_1980, unique_1980)

1980s


In [22]:
print("BAD", generate_bad_lyrics(final_1980, alpha_1980))
print()
print("OKAY", generate_okay_lyrics(final_1980, alpha_1980))

BAD 
charade lets do it all the night 
and i know i know i know 
i know i know i know i 
know i know i know i know 
i know i know i know i 
know i know i know i know 
i know i know i know i 
know i know i know i know 
i know i know i know i 
know i know i know i know 
i know i know i know i 
know i know i know i know 
i know i know i know i 
know i know i know i know 
i know i know i know i 
know i know i know i know 
i know i know i know i 
know i know i know i 

OKAY 
fevers ghostbusterswho ghostif Ì¢cause gibts gibraltar giddy 
gift i want you can i cant 
you i cant get you know what 
im not so much too much time 
to you and you can be my 
mind in love is what you and 
i dont you i can be there 
aint got me to the way i 
can i can feel you can do 
the time to you want it was 
the time is all the way you 
i know what im in the one 
who whos to do you and i 
know what to do do you i 
dont need a little thing to do 
the night long to the time and 
if you want to be your love 
is the ni

In [23]:
#1990s
print("1980s")
list_1990, unique_1990 = create_states(1990)
final_1990, alpha_1990 = create_transitions(1990, list_1990, unique_1990)

1980s


In [24]:
print("BAD", generate_bad_lyrics(final_1990, alpha_1990))
print()
print("OKAY", generate_okay_lyrics(final_1990, alpha_1990))

BAD 
average ordinary world i dont know i 
dont know i dont know i dont 
know i dont know i dont know 
i dont know i dont know i 
dont know i dont know i dont 
know i dont know i dont know 
i dont know i dont know i 
dont know i dont know i dont 
know i dont know i dont know 
i dont know i dont know i 
dont know i dont know i dont 
know i dont know i dont know 
i dont know i dont know i 
dont know i dont know i dont 
know i dont know i dont know 
i dont know i dont know i 
dont know i dont know i dont 
know i dont know i dont 

OKAY 
upon the one to be your body 
baby im a little bit taller fritos 
front huh uh yeah baby baby i 
dont you know i cant you want 
it to be the world seems to 
you and you can you can be 
my mind and im not a good 
enough baby i want no one more 
time i can you know i can 
i want no i can do you 
and you want it to be with 
me you i cant you can do 
it all the night long to be 
there is it was the one of 
my eyes of love is what you 
and i can you know you ca

In [25]:
#2000s
print("2000s")
list_2000, unique_2000 = create_states(2000)
final_2000, alpha_2000 = create_transitions(2000, list_2000, unique_2000)

2000s


In [26]:
print("BAD", generate_bad_lyrics(final_2000, alpha_2000))
print()
print("OKAY", generate_okay_lyrics(final_2000, alpha_2000))

BAD 
backlift me and i know you know 
you know you know you know you 
know you know you know you know 
you know you know you know you 
know you know you know you know 
you know you know you know you 
know you know you know you know 
you know you know you know you 
know you know you know you know 
you know you know you know you 
know you know you know you know 
you know you know you know you 
know you know you know you know 
you know you know you know you 
know you know you know you know 
you know you know you know you 
know you know you know you know 
you know you know you know 

OKAY 
painting friends when youre not the world 
to do you know that i got 
me to do it all the same 
and the one more pain and all 
of the club yea yea in a 
nigga nothin else you can see that 
we were my heart to me i 
cant be my heart to do it 
like the world and im not to 
be the way to be your mind 
she got a a man and im 
just to me in a nigga like 
you can you and the one wish 
one thing like it like a 

In [27]:
#2010s
print("2010s")
list_2010, unique_2010 = create_states(2010)
final_2010, alpha_2010 = create_transitions(2010, list_2010, unique_2010)

2010s


In [28]:
print("BAD", generate_bad_lyrics(final_2010, alpha_2010))
print()
print("OKAY", generate_okay_lyrics(final_2010, alpha_2010))

BAD 
beimma be the way you know you 
know you know you know you know 
you know you know you know you 
know you know you know you know 
you know you know you know you 
know you know you know you know 
you know you know you know you 
know you know you know you know 
you know you know you know you 
know you know you know you know 
you know you know you know you 
know you know you know you know 
you know you know you know you 
know you know you know you know 
you know you know you know you 
know you know you know you know 
you know you know you know you 
know you know you know you 

OKAY 
we dont know that i got a 
little closer winner and we dont want 
to me and the world dont care 
what i got that im a little 
love is my life oh baby dont 
know that you know that i got 
me love is your love is a 
good time that you can you want 
your eyes yeah i know how we 
dont want you know how i wanna 
know i cant see it all night 
yeah you can i got my heart 
and you can you know what do 
the only t